In [18]:
import quandl
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine

## Create Database connection

In [19]:
connection_string = "root:password@localhost/finance_db"
engine = create_engine(f'mysql://{connection_string}')

In [20]:
quandl.ApiConfig.api_key = "_ActGA_oUdxE_dxcFcQn"

## Extract CSV into DataFrame

In [21]:
fundamental_df = pd.read_csv('fundamentals.csv')
fundamental_df['Period Ending'] = pd.to_datetime(fundamental_df['Period Ending'])
fundamental_df = fundamental_df.set_index('Period Ending')
fundamental_df = fundamental_df.loc['12-01-2012':'12-31-2017']
fundamental_df.columns
fundamental_df_reduced = fundamental_df[['Ticker Symbol','For Year','Total Revenue','Total Liabilities','Total Assets','Total Equity']]
fundamental_df_reduced = fundamental_df_reduced.rename(columns={"Ticker Symbol": "Ticker",
                                                          "For Year": "Year",
                                                          "Total Revenue": "Revenue",
                                                          "Total Liabilities": "Liabilities",
                                                          "Total Assets": "Assets",
                                                          "Total Equity": "Equity"})
fundamental_df_reduced.head()

,Ticker,Year,Revenue,Liabilities,Assets,Equity
Period Ending,,,,,,
2012-12-31,AAL,2012.0,2.485500e+10,2.489100e+10,2.351000e+10,-7.987000e+09
2013-12-31,AAL,2013.0,2.674300e+10,4.500900e+10,4.227800e+10,-2.731000e+09
2014-12-31,AAL,2014.0,4.265000e+10,4.120400e+10,4.322500e+10,2.021000e+09
2015-12-31,AAL,2015.0,4.099000e+10,4.278000e+10,4.841500e+10,5.635000e+09
2012-12-29,AAP,2012.0,6.205003e+09,3.403120e+09,4.613814e+09,1.210694e+09


In [22]:
tickers = fundamental_df['Ticker Symbol'].unique().tolist()
tickers

['AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ADBE',
 'ADI',
 'ADM',
 'ADS',
 'ADSK',
 'AEE',
 'AEP',
 'AFL',
 'AIG',
 'AIV',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALK',
 'ALL',
 'ALLE',
 'ALXN',
 'AMAT',
 'AME',
 'AMG',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANTM',
 'AON',
 'APA',
 'APC',
 'APD',
 'APH',
 'ARNC',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AYI',
 'AZO',
 'BA',
 'BAC',
 'BAX',
 'BBBY',
 'BBT',
 'BBY',
 'BCR',
 'BDX',
 'BHI',
 'BIIB',
 'BK',
 'BLL',
 'BMY',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CAT',
 'CB',
 'CBG',
 'CCI',
 'CCL',
 'CELG',
 'CERN',
 'CF',
 'CFG',
 'CHD',
 'CHK',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COG',
 'COL',
 'COO',
 'COST',
 'CPB',
 'CRM',
 'CSCO',
 'CSRA',
 'CSX',
 'CTAS',
 'CTL',
 'CTSH',
 'CTXS',
 'CVS',
 'CVX',
 'CXO',
 'D',
 'DAL',
 'DD',
 'DE',
 'DFS',
 'DG',
 'DGX',
 'DHI',
 'DHR',
 'DIS',
 'DISCA',
 'DISCK',
 'DLPH',
 'DLR',
 'DLTR',
 'DNB

In [23]:
#set start date (minus 1yr to pull yr. avg price)
start_date = fundamental_df.index.min() - timedelta(days=370)

In [24]:
#set end date for pull
end_date = fundamental_df.index.max()

## Extract API into DataFrame

In [25]:
#data pull
data = quandl.get_table('WIKI/PRICES', ticker = tickers, 
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                        date = { 'gte': start_date, 'lte': end_date }, 
                        paginate=True)

In [26]:
data['Year'] = data['date'].dt.year
data

,ticker,date,adj_close,Year
None,,,,
0,ZTS,2016-12-30,53.228218,2016
1,ZTS,2016-12-29,53.317711,2016
2,ZTS,2016-12-28,53.138726,2016
3,ZTS,2016-12-27,53.417147,2016
4,ZTS,2016-12-23,53.476809,2016
5,ZTS,2016-12-22,52.959741,2016
6,ZTS,2016-12-21,52.820530,2016
7,ZTS,2016-12-20,52.800643,2016
8,ZTS,2016-12-19,52.094645,2016


In [27]:
engine.table_names()

['combined_data',
 'fundamental_stock_data',
 'year_avg_data',
 'yearly_stock_data']

## Load DataFrames into the Database

In [28]:
data.to_sql(name = 'yearly_stock_data', con=engine, if_exists='replace',index=False)
fundamental_df_reduced.to_sql(name = 'fundamental_stock_data', con=engine,if_exists='replace',index=False)

In [29]:
engine.table_names()

['combined_data',
 'fundamental_stock_data',
 'year_avg_data',
 'yearly_stock_data']